In [68]:
import pandas as pd
import numpy as np
import os
import string
os.getcwd()
import glob



In [69]:
def convert_mm(value):
    #length in mm transformed to float
    try:
        new_value = value.replace(' mm','')
        #print('old/new',float(new_value))
        return float(new_value)    
    except AttributeError:
        #print ('WTF',value)
        #return np.NaN
        return float(0)

In [88]:
def convert_exposure(value):
    #length in mm transformed to float
    
    list_string=value.split('/')
    print('listing',list_string )
    if (len(list_string)==1):
        print('length is one')
        return float(value)
    else:
        print('length is larger, now',float(list_string[0]),float(list_string[1]),float(list_string[0])/float(list_string[1]))
        return float(list_string[0])/float(list_string[1])

In [89]:
dataframe_list=[]

for input in range(2006,2019):
    df=pd.read_csv("/Users/matthiasweber/outputEXIFcsvs/PhotoEXIFDataDayType_%s.csv"%(str(input)))
    #push back year index as additional list 
    #print(df.columns,df.head)
    #print(df.dtypes)
    df.replace('/Volumes/INTENSO/Backups.backupdb/Matthias', '')
    df['SourceFile'].replace(to_replace='/Volumes/INTENSO/Backups.backupdb/Matthias',value='')    
    df['FocalLength']=df['FocalLength'].apply(convert_mm) 
    #print('before',df['ExposureTime'])
    #df['ExposureTime']=pd.to_numeric(df['ExposureTime'],errors='coerce')
    #print(df['ExposureTime'])
    df['ExposureTime']=df['ExposureTime'].apply(convert_exposure)
    df['year']=int(input)
    df.rename(columns={'DateTimeOriginal':'Date','FocalLength':'FocalLength in mm'},inplace=True) 
    #print('did something happen')

    #print(df.dtypes)
    #print('final loop')
    dataframe_list.append(df)


    
for data in dataframe_list:
     print(df.head)
    

listing ['1', '800']
length is larger, now 1.0 800.0
listing ['1', '320']
length is larger, now 1.0 320.0
listing ['1', '640']
length is larger, now 1.0 640.0
listing ['1', '125']
length is larger, now 1.0 125.0
listing ['1', '1000']
length is larger, now 1.0 1000.0
listing ['1', '640']
length is larger, now 1.0 640.0
listing ['1', '200']
length is larger, now 1.0 200.0
listing ['1', '8']
length is larger, now 1.0 8.0
listing ['1', '20']
length is larger, now 1.0 20.0
listing ['1', '25']
length is larger, now 1.0 25.0
listing ['1', '30']
length is larger, now 1.0 30.0
listing ['1', '20']
length is larger, now 1.0 20.0
listing ['1', '20']
length is larger, now 1.0 20.0
listing ['1', '6']
length is larger, now 1.0 6.0
listing ['1', '10']
length is larger, now 1.0 10.0
listing ['1', '4']
length is larger, now 1.0 4.0
listing ['1', '320']
length is larger, now 1.0 320.0
listing ['1', '400']
length is larger, now 1.0 400.0
listing ['1', '200']
length is larger, now 1.0 200.0
listing ['1', '

listing ['0.5']listing ['1', '800']
length is larger, now 1.0 800.0
listing ['1', '320']
length is larger, now 1.0 320.0
listing ['1', '640']
length is larger, now 1.0 640.0
listing ['1', '125']
length is larger, now 1.0 125.0
listing ['1', '1000']
length is larger, now 1.0 1000.0
listing ['1', '640']
length is larger, now 1.0 640.0
listing ['1', '200']
length is larger, now 1.0 200.0
listing ['1', '8']
length is larger, now 1.0 8.0
listing ['1', '20']
length is larger, now 1.0 20.0
listing ['1', '25']
length is larger, now 1.0 25.0
listing ['1', '30']
length is larger, now 1.0 30.0
listing ['1', '20']
length is larger, now 1.0 20.0
listing ['1', '20']
length is larger, now 1.0 20.0
listing ['1', '6']
length is larger, now 1.0 6.0
listing ['1', '10']
length is larger, now 1.0 10.0
listing ['1', '4']
length is larger, now 1.0 4.0
listing ['1', '320']
length is larger, now 1.0 320.0
listing ['1', '400']
length is larger, now 1.0 400.0
listing ['1', '200']
length is larger, now 1.0 200.0


listing ['0.5']
length is one
listing ['1', '80']
length is larger, now 1.0 80.0
listing ['1', '320']
length is larger, now 1.0 320.0
listing ['1', '250']
length is larger, now 1.0 250.0
listing ['1', '200']
length is larger, now 1.0 200.0
listing ['1', '100']
length is larger, now 1.0 100.0
listing ['1', '60']
length is larger, now 1.0 60.0
listing ['1', '50']
length is larger, now 1.0 50.0
listing ['1', '60']
length is larger, now 1.0 60.0
listing ['1', '30']
length is larger, now 1.0 30.0
listing ['4']
length is one
listing ['4']
length is one
listing ['1', '1000']
length is larger, now 1.0 1000.0
listing ['4']
length is one
listing ['2']
length is one
listing ['1', '50']
length is larger, now 1.0 50.0
listing ['1', '160']
length is larger, now 1.0 160.0
listing ['1', '50']
length is larger, now 1.0 50.0
listing ['1', '400']
length is larger, now 1.0 400.0
listing ['1', '100']
length is larger, now 1.0 100.0
listing ['1', '250']
length is larger, now 1.0 250.0
listing ['1', '50']
le


length is one
listing ['1', '80']
length is larger, now 1.0 80.0
listing ['1', '320']
length is larger, now 1.0 320.0
listing ['1', '250']
length is larger, now 1.0 250.0
listing ['1', '200']
length is larger, now 1.0 200.0
listing ['1', '100']
length is larger, now 1.0 100.0
listing ['1', '60']
length is larger, now 1.0 60.0
listing ['1', '50']
length is larger, now 1.0 50.0
listing ['1', '60']
length is larger, now 1.0 60.0
listing ['1', '30']
length is larger, now 1.0 30.0
listing ['4']
length is one
listing ['4']
length is one
listing ['1', '1000']
length is larger, now 1.0 1000.0
listing ['4']
length is one
listing ['2']
length is one
listing ['1', '50']
length is larger, now 1.0 50.0
listing ['1', '160']
length is larger, now 1.0 160.0
listing ['1', '50']
length is larger, now 1.0 50.0
listing ['1', '400']
length is larger, now 1.0 400.0
listing ['1', '100']
length is larger, now 1.0 100.0
listing ['1', '250']
length is larger, now 1.0 250.0
listing ['1', '50']
length is larger,

length is larger, now 1.0 500.0
listing ['1', '400']
length is larger, now 1.0 400.0
listing ['1', '200']
length is larger, now 1.0 200.0
listing ['1', '10']
length is larger, now 1.0 10.0
listing ['1', '4']
length is larger, now 1.0 4.0
listing ['1', '50']
length is larger, now 1.0 50.0
listing ['1', '5']
length is larger, now 1.0 5.0
listing ['1', '10']
length is larger, now 1.0 10.0
listing ['1', '10']
length is larger, now 1.0 10.0
listing ['0.3']
length is one
listing ['1', '40']
length is larger, now 1.0 40.0
listing ['1', '40']
length is larger, now 1.0 40.0
listing ['1', '5']
length is larger, now 1.0 5.0
listing ['0.5']
length is one
listing ['0.3']
length is one
listing ['1', '50']
length is larger, now 1.0 50.0
listing ['0.5']
length is one
listing ['0.3']
length is one
listing ['1', '4']
length is larger, now 1.0 4.0
listing ['1', '13']
length is larger, now 1.0 13.0
listing ['1', '10']
length is larger, now 1.0 10.0
listing ['1', '1000']
length is larger, now 1.0 1000.0
li

...

listing  1.0 320.0
listing ['1', '200']
length is larger, now 1.0 200.0
listing ['1', '800']
length is larger, now 1.0 800.0
listing ['1', '800']
length is larger, now 1.0 800.0
listing ['1', '250']
length is larger, now 1.0 250.0
listing ['4']
length is one
listing ['1', '320']
length is larger, now 1.0 320.0
listing ['1', '400']
length is larger, now 1.0 400.0
listing ['1', '250']
length is larger, now 1.0 250.0
listing ['1', '640']
length is larger, now 1.0 640.0
listing ['1', '200']
length is larger, now 1.0 200.0
listing ['1', '500']
length is larger, now 1.0 500.0
listing ['1', '400']
length is larger, now 1.0 400.0
listing ['1', '800']
length is larger, now 1.0 800.0
listing ['1', '60']
length is larger, now 1.0 60.0
listing ['1', '200']
length is larger, now 1.0 200.0
listing ['1', '640']
length is larger, now 1.0 640.0
listing ['1', '25']
length is larger, now 1.0 25.0
listing ['1', '10']
length is larger, now 1.0 10.0
listing ['1', '8']
length is larger, now 1.0 8.0
listing [

listing ['0.5']
length is one
listing ['0.5']
length is one
listing ['0.5']
length is one
listing ['0.5']
length is one
listing ['0.5']
length is one
listing ['4']
length is one
listing ['1', '640']
length is larger, now 1.0 640.0
listing ['1', '800']
length is larger, now 1.0 800.0
listing ['1', '640']
length is larger, now 1.0 640.0
listing ['1', '800']
length is larger, now 1.0 800.0
listing ['1', '800']
length is larger, now 1.0 800.0
listing ['1', '80']
length is larger, now 1.0 80.0
listing ['1', '125']
length is larger, now 1.0 125.0
listing ['1', '160']
length is larger, now 1.0 160.0
listing ['1', '100']
length is larger, now 1.0 100.0
listing ['1', '250']
length is larger, now 1.0 250.0
listing ['1', '125']
length is larger, now 1.0 125.0
listing ['1', '10']
length is larger, now 1.0 10.0
listing ['1', '100']
length is larger, now 1.0 100.0
listing ['1', '250']
length is larger, now 1.0 250.0
listing ['1', '20']
length is larger, now 1.0 20.0
listing ['1', '50']
length is lar

AttributeError: 'float' object has no attribute 'split'

['0.5']
length is one
listing ['0.5']
length is one
listing ['0.5']
length is one
listing ['0.5']
length is one
listing ['0.5']
length is one
listing ['4']
length is one
listing ['1', '640']
length is larger, now 1.0 640.0
listing ['1', '800']
length is larger, now 1.0 800.0
listing ['1', '640']
length is larger, now 1.0 640.0
listing ['1', '800']
length is larger, now 1.0 800.0
listing ['1', '800']
length is larger, now 1.0 800.0
listing ['1', '80']
length is larger, now 1.0 80.0
listing ['1', '125']
length is larger, now 1.0 125.0
listing ['1', '160']
length is larger, now 1.0 160.0
listing ['1', '100']
length is larger, now 1.0 100.0
listing ['1', '250']
length is larger, now 1.0 250.0
listing ['1', '125']
length is larger, now 1.0 125.0
listing ['1', '10']
length is larger, now 1.0 10.0
listing ['1', '100']
length is larger, now 1.0 100.0
listing ['1', '250']
length is larger, now 1.0 250.0
listing ['1', '20']
length is larger, now 1.0 20.0
listing ['1', '50']
length is larger, now

AttributeError: 'float' object has no attribute 'split'